<a href="https://colab.research.google.com/github/addisonpletcher/arctic_ice_dynamics/blob/main/Ice_LakeTimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
author: @ericslevenson
adapted by: @addisonpletcher
date: 10/15/2023
description: GEE script to export near-daily records of lake area within a
shapefile of buffered lakes

Copy of Lake Script, adding in Ice classification
'''

'\nauthor: @ericslevenson\nadapted by: @addisonpletcher\ndate: 10/15/2023\ndescription: GEE script to export near-daily records of lake area within a\nshapefile of buffered lakes\n\nCopy of Lake Script, adding in Ice classification\n'

# Preliminary

In [ ]:
# Earth Engine setup
import ee # Trigger the authentication flow.
ee.Authenticate()
ee.Initialize(project='ee-addisonpletcher') # Initialize the library.

# Google Drive setup (if needed)
from google.colab import drive
drive.mount('/content/drive')

#Libraries
import folium
import geemap

#### Inputs

In [35]:
## Regularly Adjusted inputs ##
# Time Period
start = '2020-04-01'
finish = '2020-06-30'
# Tile of interest
tile = 17
# Lake Shapefile
#lakes = ee.FeatureCollection('projects/ee-addisonpletcher/assets/Kusk_LakeMask') # to be used for water classification
lakes_forIce = ee.FeatureCollection('projects/ee-addisonpletcher/assets/Kusk_LakeMask') # to be used for ice classification

## Periodically Adjusted Inputs ##
# Area of Interest
aoi = ee.FeatureCollection('projects/ee-addisonpletcher/assets/Gridded_Kusk_ROI')

## Rarely Adjusted inputs ##
# Image scale
pixScale = 10

# ***EXPORTS***
# Export Properties
exportSelectorsIce = ['Lake_ID', 'iceArea', 'totalArea', 'clearArea', 'cloudArea', 'centroid']
# Description
labelI = str('tile'+str(tile))
# Export Folder
exportFolder = 'YKD_gridded'



# ***EARTH ENGINE-IFY***
eestart = ee.Date(start)
eefinish = ee.Date(finish)
startDoy = ee.Date(start).getRelative('day', 'year')
endDoy = ee.Date(finish).getRelative('day', 'year')
eeroi = aoi.filter(ee.Filter.eq('id', tile)).first()
roi = ee.Geometry.Polygon(eeroi.geometry().getInfo()['coordinates'][0])
#lakes = lakes.filterBounds(roi) # filter lakes to roi
lakes_forIce = lakes_forIce.filterBounds(roi)

# Functions
#### -> Preprocessing, Ice classification, Property Extraction + Export methods

### Image Pre-Processing Methods

In [36]:
# Mask clouds in Sentinel-2
def maskS2clouds(image):
  '''Takes an input and adds two bands: cloud mask and clear mask'''
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  clear_mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0)).rename('clear_mask')
  cloud_mask = qa.bitwiseAnd(cloudBitMask).eq(1).And(qa.bitwiseAnd(cirrusBitMask).eq(1)).rename('cloud_mask')
  return image.addBands([cloud_mask,clear_mask])

def createCloudMaskForIce(image): # @ eric - added this function
    '''Creates a cloud mask for ice analysis by inverting the clear mask'''
    cloud_mask_ed = image.select('clear_mask').Not().rename('cloud_mask_ed')
    return image.addBands(cloud_mask_ed)

# Clip image
def clip_image(image):
  '''Clips to the roi defined at the beginning of the script'''
  return image.clip(roi)

def clip2lakes_ice(image):
  '''Clips an image based on the lake boundaries'''
  return image.clip(lakes_forIce)

# Get percentile cover
def getCover(image):
  '''calculates percentage of the roi covered by the clear mask. NOTE: this function
  calls the global totPixels variable that needs to be calculated in the main script.'''
  actArea = ee.Number(image.updateMask(image.select('B2')).reduceRegion(
      reducer = ee.Reducer.count(),
      scale = 100,
      maxPixels=1e12,
      ).values().get(0)).multiply(10000)
  # calculate the perc of cover OF CLEAR PIXELS
  percCover = actArea.divide(area).multiply(100)
  # number as output
  return image.set('percCover', percCover,'actArea',actArea)

# Mosaic images by date, orbit, - basically combines images together that were taken on the same day
def mosaicBy(imcol):
  '''Takes an image collection (imcol) and creates a mosaic for each day
  Returns: An image collection of daily mosaics'''
  #return the collection as a list of images (not an image collection)
  imlist = imcol.toList(imcol.size())
  # Get all the dates as list
  def imdate(im):
    date = ee.Image(im).date().format("YYYY-MM-dd")
    return date
  all_dates = imlist.map(imdate)
  # get all orbits as list
  def orbitId(im):
    orb = ee.Image(im).get('SENSING_ORBIT_NUMBER')
    return orb
  all_orbits = imlist.map(orbitId)
  # get all spacecraft names as list
  def spacecraft(im):
    return ee.Image(im).get('SPACECRAFT_NAME')
  all_spNames = imlist.map(spacecraft)
  # this puts dates, orbits and names into a nested list
  concat_all = all_dates.zip(all_orbits).zip(all_spNames);
  # here we unnest the list with flatten, and then concatenate the list elements with " "
  def concat(el):
    return ee.List(el).flatten().join(" ")
  concat_all = concat_all.map(concat)
  # here, just get distinct combintations of date, orbit and name
  concat_unique = concat_all.distinct()
  # mosaic
  def mosaicIms(d):
    d1 = ee.String(d).split(" ")
    date1 = ee.Date(d1.get(0))
    orbit = ee.Number.parse(d1.get(1)).toInt()
    spName = ee.String(d1.get(2))
    im = imcol.filterDate(date1, date1.advance(1, "day")).filterMetadata('SPACECRAFT_NAME', 'equals', spName).filterMetadata('SENSING_ORBIT_NUMBER','equals', orbit).mosaic()
    return im.set(
        "system:time_start", date1.millis(),
        "system:date", date1.format("YYYY-MM-dd"),
        "system:id", d1)
  mosaic_imlist = concat_unique.map(mosaicIms)
  return ee.ImageCollection(mosaic_imlist)


# Apply cloud mask to other bands
def applyMask(image):
  img = image.updateMask(image.select('clear_mask'))
  return img

### Ice Classification Methods

In [37]:
def ice_classify(image):
    clear_mask = image.select('clear_mask')
    ice = image.select('B4').gte(950).rename('ice')  # greater than or equal to... threshold here
    ice = ice.updateMask(clear_mask)  # Apply clear mask to ice classification
    all = image.select('B4').gte(1).rename('all')
    return image.addBands([ice, all])


### Property Extraction + Export Methods

In [38]:
def getClearArea(lake):
  clearArea = clearAreaIm.select('area').reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry = lake.geometry(),
      scale = 10,
      maxPixels=1e9
  ).get('area')
  return clearArea

def getCloudArea(lake):
  cloudArea = cloudAreaIm.select('area').reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry = lake.geometry(),
      scale = 10,
      maxPixels=1e9
  ).get('area')
  return cloudArea

def troid(lake):
  center = ee.Array(lake.centroid().geometry().coordinates())
  return center

def getLake_ID(lake):
  '''get the Lake_ID field from shapefile'''
  Lake_ID = ee.Number(lake.get('Lake_ID'))
  return Lake_ID

def getTotalArea(lake):
  totalArea = ee.Number(lake.get('area'))
  return totalArea

def getIceArea(lake):
  iceArea = iceAreaIm.select('area').reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry = lake.geometry(),
      scale = 10,
      maxPixels=1e9
  ).get('area')
  return iceArea

def lakePropsIce(lake):
  iceArea = getIceArea(lake)
  Lake_ID = getLake_ID(lake)
  clearArea = getClearArea(lake)
  totalArea = getTotalArea(lake)
  cloudArea = getCloudArea(lake)
  centroid = troid(lake)
  return ee.Feature(None, {'Lake_ID': Lake_ID, 'iceArea': iceArea, 'clearArea': clearArea, 'totalArea': totalArea, 'cloudArea': cloudArea, 'centroid': centroid})

########################################################################

## ***EXPORT METHODS***
def export_lakes(collection, description, fileNamePrefix, fileFormat, folder, selectors):
  '''Export a feature collection of lake properties to google drive for a given day.'''
  task = ee.batch.Export.table.toDrive(**{
    'collection': collection,
    'description': description,
    'fileNamePrefix': fileNamePrefix,
    'fileFormat': fileFormat,
    'folder': folder,
    'selectors': selectors
  })
  task.start()

# Main

In [39]:
from enum import unique
##############################################################################
## *** IMAGE PROCESSING ***
images = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterBounds(roi).filterDate(start,finish).filter(ee.Filter.calendarRange(startDoy, endDoy, 'day_of_year')).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',80)) # Get Images
images_all = mosaicBy(images) # Mosaic images
images_all = images_all.map(maskS2clouds) # Create cloud/clear masks
images_all = images_all.map(createCloudMaskForIce) # @ eric - I added this
images_all = images_all.map(clip_image) # Clip to roi
images_all.map(applyMask) # mask other bands for clouds
area = roi.area().getInfo() # Calculate total area

# Filter by percentile cover
images_all = images_all.map(getCover) # Add percent cover as an image property
images_all = images_all.filterMetadata('percCover','greater_than',10) # remove images covering less than 10% of the ROI)
lakeimages_ice = images_all.map(clip2lakes_ice) # Clip images to lake mask

#see how many images we've got
dates = lakeimages_ice.aggregate_array('system:date').getInfo()
dates

['2020-04-05',
 '2020-04-07',
 '2020-04-10',
 '2020-04-15',
 '2020-04-20',
 '2020-04-22',
 '2020-04-24',
 '2020-04-25',
 '2020-04-27',
 '2020-04-29',
 '2020-04-30',
 '2020-05-02',
 '2020-05-04',
 '2020-05-05',
 '2020-05-07',
 '2020-05-09',
 '2020-05-10',
 '2020-05-14',
 '2020-05-15',
 '2020-05-17',
 '2020-05-19',
 '2020-05-22',
 '2020-05-24',
 '2020-05-25',
 '2020-05-27',
 '2020-05-29',
 '2020-05-30',
 '2020-06-01',
 '2020-06-03',
 '2020-06-08',
 '2020-06-09',
 '2020-06-13',
 '2020-06-14',
 '2020-06-16',
 '2020-06-18',
 '2020-06-19',
 '2020-06-21',
 '2020-06-26',
 '2020-06-29']

In [40]:
###############################################################################
## ***ICE CLASSIFICATION***
lakeimages_ice = lakeimages_ice.map(ice_classify)

###############################################################################
## ***ITERATE THROUGH DAYS AND EXPORT***
for i, date in enumerate(dates):
  # Get lake properties
  eedate = ee.Date(date) #earthengine date format

  #ice
  lakeIm2 = lakeimages_ice.filterDate(eedate).first() #get date
  areaIm2 = lakeIm2.pixelArea() # get a pixel area image
  lakeIm2 = lakeIm2.addBands([areaIm2]) # add pixel area as band in image
  iceAreaIm = areaIm2.updateMask(lakeIm2.select('ice')) #mask area image based on ice
  cloudAreaIm = areaIm2.updateMask(lakeIm2.select('cloud_mask_ed')) # mask area image based on clouds
  clearAreaIm = areaIm2.updateMask(lakeIm2.select('clear_mask'))

  #apply
  ice = lakes_forIce.map(lakePropsIce)

  # Export
  exportDate = date.replace('-', '_')
  descriptionI = labelI +'_'+ exportDate
  fileformat = 'CSV'
  export_lakes(ice, descriptionI, descriptionI, fileformat, exportFolder, exportSelectorsIce)

In [41]:
icetest = ice.first()
icetest.getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '00000000000000003353',
 'properties': {'Lake_ID': 0,
  'centroid': [-164.10687221463732, 60.345317220464445],
  'clearArea': 0,
  'cloudArea': 0,
  'iceArea': 0,
  'totalArea': 25766.61993408203}}

In [42]:
# print(lakeIm2.bandNames().getInfo())

# Temperature Data

In [43]:
# # Acquire daily mean 2m air temperature
# era5_2mt = (
#     ee.ImageCollection('ECMWF/ERA5/DAILY')
#     .select('mean_2m_air_temperature')
#     .filter(ee.Filter.date(start, finish))
# )

# #spatial join with lake dataset
# def extract_and_create_feature(date_image):
#     date = date_image.get('system:time_start')

#     def extract_temp_for_lake(lake_feature):
#         # Extract mean temperature in Kelvin
#         mean_temp_kelvin = ee.Number(date_image.reduceRegion(
#             reducer=ee.Reducer.mean(),
#             geometry=lake_feature.geometry(),
#             scale=pixScale
#         ).get('mean_2m_air_temperature'))

#         # Convert temperature from Kelvin to Celsius
#         mean_temp_celsius = mean_temp_kelvin.subtract(273.15)

#         Lake_ID = lake_feature.get('Lake_ID')

#         # Create a new feature with Lake_ID, date, and mean temperature in Celsius
#         return ee.Feature(None, {
#             'Lake_ID': Lake_ID,
#             'date': ee.Date(date).format('YYYY-MM-dd'),
#             'mean_temperature_kelvin': mean_temp_kelvin,  # Optional: keep the Kelvin value
#             'mean_temperature_celsius': mean_temp_celsius
#         })

#     # Map over each lake to extract temperature and create a new feature
#     new_features = lakes_forIce.map(extract_temp_for_lake)
#     return new_features


# ### apply function to temp dataset ###
# # Map function
# temp_features_by_date = era5_2mt.map(extract_and_create_feature)
# # Flatten into a single FeatureCollection
# flattened_features = temp_features_by_date.flatten()


In [44]:
# def export_temperature_data(collection, description, fileNamePrefix, folder):
#     task = ee.batch.Export.table.toDrive(**{
#         'collection': collection,
#         'description': description,
#         'fileNamePrefix': fileNamePrefix,
#         'folder': folder,
#         'fileFormat': 'CSV'
#     })
#     task.start()

# # Call the export function
# export_description = 'Lake_Temperature_Data'
# start_year = start.split('-')[0]
# export_fileNamePrefix = 'Lake_Temperature_' + start_year
# export_folder_T = 'Kusk_TempData'

# # export_temperature_data(flattened_features, export_description, export_fileNamePrefix, export_folder_T)


#### Test

In [45]:
# # Take a sample of 10 features from the flattened collection
# sample_features = flattened_features.limit(10)

# # Print the sample to the console
# print('Sample Features:', sample_features.getInfo())


# Visualization


## Map
##### Prelim. map steps

In [46]:
# lakeimages_ice = lakeimages_ice.map(ice_classify)
# iceimg = ee.Image(lakeimages_ice.first()) #isolate single image
# iceimg.getInfo() #forcing GEE in order to visualize


### Mapping

In [47]:
# Map = geemap.Map()
# Map.centerObject(roi, 10)
# Map.addLayer(iceimg.select('clear_mask'), {'palette': ['grey', 'white']}, 'Clear Mask')
# #Map.addLayer(iceimg.select('cloud_mask_ed'), {'palette': ['grey', 'white']}, 'Clouds')
# Map.addLayer(iceimg.select('ice'), {'palette': ['white', 'orange']}, 'Ice Classification')
# Map.addLayer(roi)
# Map.addLayerControl()
# Map

In [48]:
# #TEST
# sample_collection = ee.ImageCollection('COPERNICUS/S2').filterBounds(roi).filterDate(start,finish)

# classified_collection = sample_collection.map(ice_classify)
# classified_collection = classified_collection.map(maskS2clouds) #'''not working so removed'''
# classified_image = ee.Image(classified_collection.first())
# classified_image = classified_image.clip(lakes_forIce)

# #VISUALIZE
# Map = geemap.Map()
# Map.centerObject(roi, 10)
# Map.addLayer(classified_image.select('ice'), {'palette': ['blue', 'white']}, 'Ice Classification')
# Map.addLayer(classified_image.select('cloud_mask'), {'palette': ['red'], 'min': 0, 'max': 1}, 'Cloud Mask')  # Add cloud mask as a layer
# Map.addLayer(classified_image.select('clear_mask'), {'palette': ['green'], 'min': 0, 'max': 1}, 'Clear Mask')  # Add clear mask as a layer
# #Map.addLayer(classified_image.select('all'), {'palette': ['gray']}, 'All Bands')
# #Map.addLayer(roi, {'color': 'red'}, 'ROI')
# Map.addLayer(lakes_forIce,{'color': 'orange'}, 'Lake Mask (unbuffered)')
# Map.addLayerControl()
# Map

## Classified Image Export (single, collection)

### Single Image

In [49]:
# #Original visualization
# single = lakeimages_ice.first().select('ice', 'clear_mask')

# # Cast all bands to UInt 8
# single = single.toUint8() #exporting image w/ binary classifications ranging from 0-1, so UInt8

# task = ee.batch.Export.image.toDrive(**{
#     'image': single,
#     'description': 'test_img',
#     'folder':'Visualization',
#     'fileFormat': 'GeoTIFF',
#     'scale': 10,
#     'region': roi,
#     'maxPixels': 1e12
# })
# task.start()
# import time
# while task.active():
#   print('Polling for task (id: {}).'.format(task.id))
#   time.sleep(5)

### Image Collection

In [50]:
# import folium
# import time

# # Define what bands
# bands_to_select = ['water_75', 'cloud_mask', 'clear_mask']

# # Iterate through the Image Collection and export each image
# image_list = lakeimages.toList(lakeimages.size())
# for i in range(image_list.length().getInfo()):
#     # Get the current image in the collection
#     current_image = ee.Image(image_list.get(i))

#     # Select the desired bands and cast to UInt8
#     selected_image = current_image.select(bands_to_select).toUint8()

#     # Get the date from the image's metadata
#     date = ee.Date(current_image.get('system:time_start'))
#     date_str = date.format('YYYY-MM-dd').getInfo()  # Format the date as desired

#     # Use the date as the description
#     description = 'image_' + date_str

#     # Define the export task for the current image
#     task = ee.batch.Export.image.toDrive(**{
#         'image': selected_image,
#         'description': description,
#         'folder': 'Visualization', #have this foler be mapped within YKD_Water folder
#         'fileFormat': 'GeoTIFF',
#         'scale': 10,
#         'region': roi,
#         'maxPixels': 1e12
#     })

#     # Start the export task for the current image
#     task.start()

#     # Monitor the task's progress
#     print('Exporting task (id: {}) - Image {}'.format(task.id, i))

#     # Wait for the task to complete before moving on to the next image
#     while task.active():
#         print('Polling for task (id: {}).'.format(task.id))
#         time.sleep(5)
